In [1]:
import pandas as pd
import numpy as np
import sqlite3 as db
from datetime import datetime
import urllib
import os

from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager

import time

In [3]:


driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())




====== WebDriver manager ======
Current firefox version is 102.0
Get LATEST geckodriver version for 102.0 firefox
Driver [C:\Users\egonu\.wdm\drivers\geckodriver\win64\v0.31.0\geckodriver.exe] found in cache
C:\Users\egonu\AppData\Local\Temp/ipykernel_33212/117907651.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())


In [4]:

bigmacLink = "https://siparis.mcdonalds.com.tr/productv2/?pid=41"

driver.set_page_load_timeout(20)

try:
    driver.get(bigmacLink)
except:
    print("timeout")
    time.sleep(2)

html = driver.page_source

# close web browser
driver.close()

In [5]:

from pyquery import PyQuery  

pq = PyQuery(html)


priceTxt = pq(".item-price").text().replace(",",".")

price = float(priceTxt)

In [6]:

con = db.connect(database="../db.sqlite")
cursor = con.cursor()

cursor.execute("""select LocalPrice from BigMacPrice
                where Country_id = 'TUR'
                order by Date desc
                limit 1""")




latestPrice = cursor.fetchall()[0][0]
print(latestPrice)


if price != latestPrice:    
    print("different price")
    print(price)
    
    cursor.execute("""select Close from CurrencyUSDValue
                    where Currency_id = 'TRY'
                    order by Date desc
                    limit 1""")
    
    currClose = cursor.fetchall()[0][0]
    
    print(currClose)
    
    usdPrice = price / currClose
    
    print(usdPrice)
    
    
    cursor.execute("""insert into BigMacPrice(Country_id, Date, LocalPrice, USDPrice, Source) Values(?,?,?,?,?)""",
                    ("TUR",datetime.today(),price,usdPrice,"mc donalds"))
    con.commit()

50.0
different price
13.2905
4.288777698355968
